In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("/Users/Abhirup/Desktop/Projects/HSBC DS Assignment/PS_20174392719_1491204439457_log.csv")
df.sort_values(by=['step'])
df = df.rename(columns={'oldbalanceOrg':'oldBalanceOrig', 'newbalanceOrig':'newBalanceOrig', \
                        'oldbalanceDest':'oldBalanceDest', 'newbalanceDest':'newBalanceDest'})
df.head()

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [4]:
print(df.shape)
print(df.dtypes)

(6362620, 11)
step                int64
type               object
amount            float64
nameOrig           object
oldBalanceOrig    float64
newBalanceOrig    float64
nameDest           object
oldBalanceDest    float64
newBalanceDest    float64
isFraud             int64
isFlaggedFraud      int64
dtype: object


In [5]:
df['step'].value_counts().sort_index()

1      2708
2      1014
3       552
4       565
5       665
       ... 
739      10
740       6
741      22
742      14
743       8
Name: step, Length: 743, dtype: int64

In [6]:
df['step'].value_counts(bins=4).sort_index()

(0.257, 186.5]    2247887
(186.5, 372.0]    3038001
(372.0, 557.5]     882793
(557.5, 743.0]     193939
Name: step, dtype: int64

In [7]:
#Since Steps are hours, make bins of observations to weeks
for i in range(4):
    df.loc[(df['step']>186*i) & (df['step']<=186*(i+1)),'week'] =i+1
df.head()

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud,week
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0,1.0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0,1.0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0,1.0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0,1.0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0,1.0


In [8]:
print(df['week'].value_counts().sort_index())
print(df.groupby('week')['isFraud'].sum())

1.0    2247887
2.0    3038001
3.0     882807
4.0     193925
Name: week, dtype: int64
week
1.0    2122
2.0    2031
3.0    2012
4.0    2048
Name: isFraud, dtype: int64


#### Findings1: Frauds are evenly distributed across weeks

In [9]:
df.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldBalanceOrig    0
newBalanceOrig    0
nameDest          0
oldBalanceDest    0
newBalanceDest    0
isFraud           0
isFlaggedFraud    0
week              0
dtype: int64

#### Findings 2: No missing Value in Data

In [10]:
print(df.groupby('week')['nameOrig'].count())
print(df.groupby('week')['nameOrig'].nunique())

week
1.0    2247887
2.0    3038001
3.0     882807
4.0     193925
Name: nameOrig, dtype: int64
week
1.0    2246734
2.0    3035880
3.0     882609
4.0     193920
Name: nameOrig, dtype: int64


In [11]:
for i in range(1,4):
    print("\nCustomers Transacting both in Week %s and Week %s"%(i,i+1))
    print(pd.merge(df.loc[df['week']==i,'nameOrig'],df.loc[df['week']==i+1,'nameOrig'],on='nameOrig')
      ['nameOrig'].nunique())
    print("\nCustomers Transacting both in Week %s and Week %s"%(i,i+2))
    print(pd.merge(df.loc[df['week']==i,'nameOrig'],df.loc[df['week']==i+2,'nameOrig'],on='nameOrig')
      ['nameOrig'].nunique())


Customers Transacting both in Week 1 and Week 2
3244

Customers Transacting both in Week 1 and Week 3
865

Customers Transacting both in Week 2 and Week 3
1185

Customers Transacting both in Week 2 and Week 4
270

Customers Transacting both in Week 3 and Week 4
73

Customers Transacting both in Week 3 and Week 5
0


#### Findings 3: not many customers repeating transactions every week. historical me vs me variables will be sparse

In [12]:
common_cust_df=pd.DataFrame(columns=['nameOrig'])
for i in range(1,4):
    common_cust_df=common_cust_df.append(pd.merge(df.loc[df['week']==i,'nameOrig'],df.loc[df['week']==i+1,'nameOrig'],
                                                  on='nameOrig'))
                                
common_cust_list = common_cust_df['nameOrig'].tolist()
print("\nFraud Instances of Common Customers across Weeks")
print(df[df['nameOrig'].isin(common_cust_list)].groupby('week')['isFraud'].sum())


Fraud Instances of Common Customers across Weeks
week
1.0    1
2.0    3
3.0    3
4.0    0
Name: isFraud, dtype: int64


#### Findings 4: less fraud among repeat transacting customers

In [13]:
df.groupby(['week','type'])['isFraud'].sum()

week  type    
1.0   CASH_IN        0
      CASH_OUT    1069
      DEBIT          0
      PAYMENT        0
      TRANSFER    1053
2.0   CASH_IN        0
      CASH_OUT    1017
      DEBIT          0
      PAYMENT        0
      TRANSFER    1014
3.0   CASH_IN        0
      CASH_OUT    1006
      DEBIT          0
      PAYMENT        0
      TRANSFER    1006
4.0   CASH_IN        0
      CASH_OUT    1024
      DEBIT          0
      PAYMENT        0
      TRANSFER    1024
Name: isFraud, dtype: int64

#### Findings 5: only CASH_OUT and TRANSFER have Fraud

In [14]:
print(df.groupby(['isFraud'])['isFlaggedFraud'].sum())
print(df.loc[(df['isFraud']==0) & (df['isFlaggedFraud']==1),:].shape)

isFraud
0     0
1    16
Name: isFlaggedFraud, dtype: int64
(0, 12)


In [15]:
df[df['isFlaggedFraud']==1]['type'].value_counts()

TRANSFER    16
Name: type, dtype: int64

#### Findings 6: isFlaggedFraud is only 1 when isFraud is 1 and type is TRANSFER

In [16]:
pd.set_option('float_format', '{:f}'.format)
df[df['isFlaggedFraud']==1]['amount'].describe()

count         16.000000
mean     4861597.730625
std      3572499.102347
min       353874.220000
25%      2242748.722500
50%      4234245.095000
75%      7883451.380000
max     10000000.000000
Name: amount, dtype: float64

In [17]:
df.loc[(df['amount']>=353874.22) & (df['type']=='TRANSFER'),'isFraud'].value_counts()

0    322893
1      2246
Name: isFraud, dtype: int64

#### Findings 7: all transactions above the minimum amount when isFlaggedFraud==1 is not Fraud

In [18]:
print('\nType of transactions where isFlaggedFraud=1')
print((df.loc[df['isFlaggedFraud']==1,'type']).value_counts())
print('\nMax amount transacted in a TRANSFER where isFlaggedFraud is 0')
print(df.loc[(df['type'] == 'TRANSFER') & (df['isFlaggedFraud'] == 0),'amount'].max())


Type of transactions where isFlaggedFraud=1
TRANSFER    16
Name: type, dtype: int64

Max amount transacted in a TRANSFER where isFlaggedFraud is 0
92445516.64


#### Findings 8: isFlaggedFraud definition is not consistent with the definition provided

In [19]:
df[df['isFlaggedFraud']==1]

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud,week
2736446,212,TRANSFER,4953893.080000,C728984460,4953893.080000,4953893.080000,C639921569,0.000000,0.000000,1,1,2.000000
3247297,250,TRANSFER,1343002.080000,C1100582606,1343002.080000,1343002.080000,C1147517658,0.000000,0.000000,1,1,2.000000
3760288,279,TRANSFER,536624.410000,C1035541766,536624.410000,536624.410000,C1100697970,0.000000,0.000000,1,1,2.000000
5563713,387,TRANSFER,4892193.090000,C908544136,4892193.090000,4892193.090000,C891140444,0.000000,0.000000,1,1,3.000000
5996407,425,TRANSFER,10000000.000000,C689608084,19585040.370000,19585040.370000,C1392803603,0.000000,0.000000,1,1,3.000000
5996409,425,TRANSFER,9585040.370000,C452586515,19585040.370000,19585040.370000,C1109166882,0.000000,0.000000,1,1,3.000000
6168499,554,TRANSFER,3576297.100000,C193696150,3576297.100000,3576297.100000,C484597480,0.000000,0.000000,1,1,3.000000
6205439,586,TRANSFER,353874.220000,C1684585475,353874.220000,353874.220000,C1770418982,0.000000,0.000000,1,1,4.000000
6266413,617,TRANSFER,2542664.270000,C786455622,2542664.270000,2542664.270000,C661958277,0.000000,0.000000,1,1,4.000000
6281482,646,TRANSFER,10000000.000000,C19004745,10399045.080000,10399045.080000,C1806199534,0.000000,0.000000,1,1,4.000000


#### Findings 9: For all isFlaggedFraud==1, oldBalanceDest=newBalanceDest=0. No consistency in other variables from the snapshot above

In [20]:
len(df.loc[(df['type'] == 'TRANSFER') & (df['isFlaggedFraud'] == 0) 
           & (df['oldBalanceDest'] == 0) & (df['newBalanceDest'] == 0)])

4158

#### Findings 10: There are other TRANSFERS where isFlaggedFraud==0 and oldBalanceDest=newBalanceDest=0

In [21]:
#Creating feature Amount/Origination Old Balance
df.loc[df['oldBalanceOrig']>0,'amt_bal_ratio']=df['amount']/df['oldBalanceOrig']
df.loc[df['oldBalanceOrig']==0,'amt_bal_ratio']= np.nan
df['amt_bal_ratio'].describe()

count   4260171.000000
mean        122.040104
std        4149.277081
min           0.000000
25%           0.074596
50%           0.740758
75%           6.655752
max     3925475.600000
Name: amt_bal_ratio, dtype: float64

In [22]:
print ("\nMax and Min Amount/Origination Old Balance ratio when isFlaggedFraud==1:")
print(df.loc[df['isFlaggedFraud'] == 1,'amt_bal_ratio'].max())
print(df.loc[df['isFlaggedFraud'] == 1,'amt_bal_ratio'].min())
print ("\nMax and Min Amount/Origination Old Balance ratio when isFlaggedFraud==1 and TRANSFER and oldBalanceOrig>0:")
print(df.loc[(df['isFlaggedFraud'] == 0) & (df['type'] == 'TRANSFER') 
             & (df['oldBalanceOrig']>0),'amt_bal_ratio'].max())
print(round(df.loc[(df['isFlaggedFraud'] == 0) & (df['type'] == 'TRANSFER') 
             & (df['oldBalanceOrig']>0),'amt_bal_ratio'].min()))


Max and Min Amount/Origination Old Balance ratio when isFlaggedFraud==1:
1.0
0.03837324263238986

Max and Min Amount/Origination Old Balance ratio when isFlaggedFraud==1 and TRANSFER and oldBalanceOrig>0:
1854642.67
0


#### Findings11: No particular threshold for amt_bal_ratio when isFlaggedFraud==1. Range overlaps when isFlaggedFraud==0

In [23]:
print(len(df.loc[df['nameDest'].str[0]=='M']))
print(len(df.loc[df['nameOrig'].str[0]=='M']))

2151495
0


#### Findings 12: Merchants are only in Destination Name

In [24]:
print(df.loc[df['nameDest'].str[0]=='M','type'].value_counts())
print(df.loc[df['nameDest'].str[0]=='M','isFraud'].value_counts())

PAYMENT    2151495
Name: type, dtype: int64
0    2151495
Name: isFraud, dtype: int64


#### Findings 13: Only PAYMENT is done towards Merchants and no Fraud

In [25]:
print(df['nameOrig'].str[0].value_counts())
print(df['nameDest'].str[0].value_counts())

C    6362620
Name: nameOrig, dtype: int64
C    4211125
M    2151495
Name: nameDest, dtype: int64


#### Findings14: Only Customer in Transaction Origin and Customer and Merchant in Transaction Destination. All Frauds is happening in Transactions between Customers

In [26]:
df[df['amt_bal_ratio']>1].head(10)

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud,week,amt_bal_ratio
8,1,PAYMENT,4024.360000,C1265012928,2671.000000,0.000000,M1176932104,0.000000,0.000000,0,0,1.000000,1.506687
10,1,DEBIT,9644.940000,C1900366749,4465.000000,0.000000,C997608398,10845.000000,157982.120000,0,0,1.000000,2.160121
13,1,PAYMENT,11633.760000,C1716932897,10127.000000,0.000000,M801569151,0.000000,0.000000,0,0,1.000000,1.148786
15,1,CASH_OUT,229133.940000,C905080434,15325.000000,0.000000,C476402209,5083.000000,51513.440000,0,0,1.000000,14.951644
16,1,PAYMENT,1563.820000,C761750706,450.000000,0.000000,M1731217984,0.000000,0.000000,0,0,1.000000,3.475156
19,1,TRANSFER,215310.300000,C1670993182,705.000000,0.000000,C1100439041,22425.000000,0.000000,0,0,1.000000,305.404681
24,1,TRANSFER,311685.890000,C1984094095,10835.000000,0.000000,C932583850,6267.000000,2719172.890000,0,0,1.000000,28.766580
25,1,PAYMENT,6061.130000,C1043358826,443.000000,0.000000,M1558079303,0.000000,0.000000,0,0,1.000000,13.682009
28,1,PAYMENT,8901.990000,C1632497828,2958.910000,0.000000,M33419717,0.000000,0.000000,0,0,1.000000,3.008537
36,1,PAYMENT,8603.420000,C1608633989,253.000000,0.000000,M1615617512,0.000000,0.000000,0,0,1.000000,34.005613


In [27]:
df[df['amt_bal_ratio']>1]['newBalanceOrig'].describe()

count   1976647.000000
mean      56032.342897
std      125606.619465
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max     1753988.800000
Name: newBalanceOrig, dtype: float64

In [28]:
print("\n# Transactions where Amount > Old Balance Origin with New Balance Origin==0")
print(df.loc[(df['amt_bal_ratio']>1)&(df['newBalanceOrig']==0),'step'].count())
print("\n# Transactions where Amount > Old Balance Origin with New Balance Origin not equal to 0")
print(df.loc[(df['amt_bal_ratio']>1)&(df['newBalanceOrig']!=0),'step'].count())


# Transactions where Amount > Old Balance Origin with New Balance Origin==0
1512573

# Transactions where Amount > Old Balance Origin with New Balance Origin not equal to 0
464074


In [29]:
print("\n# Type of Transaction Didtribution where Amount > Old Balance Origin with New Balance Origin==0")
print(df.loc[(df['amt_bal_ratio']>1)&(df['newBalanceOrig']==0),'type'].value_counts())
print("\n# Type of Transaction Didtribution where Amount > Old Balance Origin with New Balance Origin not equal to 0")
print(df.loc[(df['amt_bal_ratio']>1)&(df['newBalanceOrig']!=0),'type'].value_counts())


# Type of Transaction Didtribution where Amount > Old Balance Origin with New Balance Origin==0
CASH_OUT    955337
PAYMENT     326879
TRANSFER    224729
DEBIT         5628
Name: type, dtype: int64

# Type of Transaction Didtribution where Amount > Old Balance Origin with New Balance Origin not equal to 0
CASH_IN     464072
CASH_OUT         2
Name: type, dtype: int64


#### Findings 15: New Balance Orig is not equal to zero when Amount > Old Balance Origin due to CASH IN

In [30]:
# Explore the 2 Cash Out Cases
df.loc[(df['amt_bal_ratio']>1)&(df['newBalanceOrig']!=0)&(df['type']=='CASH_OUT')]

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud,week,amt_bal_ratio
34076,8,CASH_OUT,88553.310000,C319673957,67106.460000,88553.310000,C932634391,67106.460000,0.000000,0,0,1.000000,1.319594
34779,8,CASH_OUT,106659.840000,C638668928,80084.580000,106659.840000,C224342296,80084.580000,92322.890000,0,0,1.000000,1.331840


In [31]:
df[df['nameOrig'].isin(['C319673957','C638668928'])].sort_values(by=['step'])

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud,week,amt_bal_ratio
34076,8,CASH_OUT,88553.310000,C319673957,67106.460000,88553.310000,C932634391,67106.460000,0.000000,0,0,1.000000,1.319594
34779,8,CASH_OUT,106659.840000,C638668928,80084.580000,106659.840000,C224342296,80084.580000,92322.890000,0,0,1.000000,1.331840


In [32]:
df[df['nameDest'].isin(['C319673957','C638668928'])].sort_values(by=['step'])

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud,week,amt_bal_ratio


#### Findings 16: New Balance Orig is not equal to zero and Amount > Old Balance Origin and type==CASH OUT data does not tally with Balance movements. There is not corresponding CASH IN or Name in Destination for the corresponding Names

In [33]:
df['diffBalanceOrig']=df['oldBalanceOrig']+df['amount']-df['newBalanceOrig']
df.loc[(df['amt_bal_ratio']>1)&(df['newBalanceOrig']!=0)&(df['type']=='CASH_IN'),'diffBalanceOrig'].describe()

count   464072.000000
mean         0.000002
std          0.001131
min         -0.010000
25%          0.000000
50%          0.000000
75%          0.000000
max          0.010000
Name: diffBalanceOrig, dtype: float64

In [34]:
df.loc[(df['type']=='CASH_IN'),'diffBalanceOrig'].describe()

count   1399284.000000
mean          5.038225
std        1319.894982
min          -0.010000
25%           0.000000
50%           0.000000
75%           0.000000
max      770537.370000
Name: diffBalanceOrig, dtype: float64

In [35]:
df.loc[(df['type']=='CASH_IN')&(df['diffBalanceOrig']>=1),:]

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud,week,amt_bal_ratio,diffBalanceOrig
456,1,CASH_IN,110226.340000,C1475192960,1601450.630000,1601450.630000,C1816757085,1601450.630000,10681238.790000,0,0,1.000000,0.068829,110226.340000
656,1,CASH_IN,770537.370000,C2015999862,8499043.130000,8499043.130000,C1883840933,8499043.130000,16874643.090000,0,0,1.000000,0.090662,770537.370000
1292,1,CASH_IN,61505.910000,C1790387225,4910403.440000,4910403.440000,C11003494,4910403.440000,10588885.270000,0,0,1.000000,0.012526,61505.910000
1293,1,CASH_IN,404167.600000,C984412970,4910403.440000,4910403.440000,C11003494,4910403.440000,10588885.270000,0,0,1.000000,0.082308,404167.600000
2129,1,CASH_IN,42012.450000,C1137667747,350501.240000,350501.240000,C667346055,350501.240000,9291619.620000,0,0,1.000000,0.119864,42012.450000
2246,1,CASH_IN,187256.620000,C1106468732,521018.570000,521018.570000,C662736689,521018.570000,4891090.560000,0,0,1.000000,0.359405,187256.620000
2678,1,CASH_IN,208664.250000,C1216611732,4800759.180000,4800759.180000,C1870252780,4800759.180000,7416184.790000,0,0,1.000000,0.043465,208664.250000
2694,1,CASH_IN,301755.980000,C1544031987,6509377.510000,6509377.510000,C1870252780,6509377.510000,7416184.790000,0,0,1.000000,0.046357,301755.980000
2751,2,CASH_IN,305762.250000,C690051320,1957713.690000,1957713.690000,C1778801068,1957713.690000,8813776.610000,0,0,1.000000,0.156183,305762.250000
2923,2,CASH_IN,275712.540000,C1378612814,6269058.270000,6269058.270000,C1122118227,6269058.270000,11073556.180000,0,0,1.000000,0.043980,275712.540000


#### Findings 17: For CASH IN Transactions majority cases NEW Balance Origin - OLD Balance Origin == Amount. For the cases its doesnt hold true, NEW Balance Origin == OLD Balance Origin

In [36]:
# Check Fraud nameDest
df_fraudcust_cnt=df.loc[df['isFraud']==1,['nameDest','isFraud']].groupby('nameDest',as_index=False).sum()
df_fraudcust_cnt=df_fraudcust_cnt.rename(columns={'isFraud':'Trans_Num'})
df_fraudcust_cnt=df_fraudcust_cnt.sort_values(by=['Trans_Num'],ascending=False)
df_fraudcust_cnt.head()
#df_fraudcust_cnt[df_fraudcust_cnt['Trans_Num']>1]

,nameDest,Trans_Num
4244,C2020337583,2
6699,C650699445,2
5937,C475338087,2
6065,C505532836,2
759,C1185292292,2


In [37]:
repeat_fraud_destcust = df_fraudcust_cnt[df_fraudcust_cnt['Trans_Num']>1]['nameDest'].tolist()
df[df['nameDest'].isin(repeat_fraud_destcust)].sort_values(by=['nameDest','step'])

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud,week,amt_bal_ratio,diffBalanceOrig
49826,9,CASH_OUT,2539898.070000,C728718059,2539898.070000,0.000000,C1013511446,968.000000,1842864.220000,1,0,1.000000,1.000000,5079796.140000
67782,9,CASH_IN,594209.730000,C1888557649,3151936.520000,3746146.250000,C1013511446,2540866.070000,1842864.220000,0,0,1.000000,0.188522,0.000000
83389,10,CASH_IN,103792.120000,C792601931,42696.840000,146488.970000,C1013511446,1946656.340000,1842864.220000,0,0,1.000000,2.430909,-0.010000
146724,12,CASH_OUT,398361.070000,C1402841545,8287.000000,0.000000,C1013511446,0.000000,2006826.390000,0,0,1.000000,48.070601,406648.070000
147414,12,TRANSFER,1608465.320000,C657294376,10678.000000,0.000000,C1013511446,398361.070000,2006826.390000,0,0,1.000000,150.633576,1619143.320000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4473849,324,CASH_IN,529863.480000,C924297599,4813001.110000,5342864.590000,C967226405,2903277.850000,2373414.370000,0,0,2.000000,0.110090,0.000000
4621225,329,CASH_OUT,116391.330000,C1882477120,0.000000,0.000000,C967226405,2373414.370000,2489805.700000,0,0,2.000000,nan,116391.330000
5496752,380,TRANSFER,34672.080000,C142316438,0.000000,0.000000,C967226405,2489805.700000,2524477.780000,0,0,3.000000,nan,34672.080000
5889515,403,TRANSFER,140653.400000,C1943053207,0.000000,0.000000,C967226405,2524477.780000,2665131.180000,0,0,3.000000,nan,140653.400000


In [38]:
df[df['nameDest'].isin(repeat_fraud_destcust) & df['isFraud']==1].sort_values(by=['nameDest','step'])

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud,week,amt_bal_ratio,diffBalanceOrig
49826,9,CASH_OUT,2539898.070000,C728718059,2539898.070000,0.000000,C1013511446,968.000000,1842864.220000,1,0,1.000000,1.000000,5079796.140000
1030590,74,CASH_OUT,455074.130000,C695129655,455074.130000,0.000000,C1013511446,9237170.740000,9692244.860000,1,0,1.000000,1.000000,910148.260000
788647,39,TRANSFER,152042.160000,C1118399210,152042.160000,0.000000,C104038589,0.000000,0.000000,1,0,1.000000,1.000000,304084.320000
1030368,59,CASH_OUT,2050727.810000,C992917413,2050727.810000,0.000000,C104038589,0.000000,2050727.810000,1,0,1.000000,1.000000,4101455.620000
2122590,183,CASH_OUT,224378.100000,C1234321469,224378.100000,0.000000,C105660024,483419.250000,707797.360000,1,0,1.000000,1.000000,448756.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006043,46,CASH_OUT,71372.000000,C313736303,71372.000000,0.000000,C935310781,613464.040000,684836.040000,1,0,1.000000,1.000000,142744.000000
4058811,300,CASH_OUT,176059.160000,C1203474575,176059.160000,0.000000,C964377943,105783.210000,281842.360000,1,0,2.000000,1.000000,352118.320000
5801852,401,CASH_OUT,5838803.230000,C388640141,5838803.230000,0.000000,C964377943,471758.210000,6310561.440000,1,0,3.000000,1.000000,11677606.460000
1832574,163,CASH_OUT,762135.720000,C1088995906,762135.720000,0.000000,C967226405,47768.780000,809904.500000,1,0,1.000000,1.000000,1524271.440000


In [42]:
df[(df['nameDest']=='C1013511446') | (df['nameOrig']=='C1013511446')].sort_values(by='step')
# Why C1013511446 is receiving money post Fraud? In step 12 and 13 balance movements are not similar to each other

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud,week,amt_bal_ratio,diffBalanceOrig
49826,9,CASH_OUT,2539898.070000,C728718059,2539898.070000,0.000000,C1013511446,968.000000,1842864.220000,1,0,1.000000,1.000000,5079796.140000
67782,9,CASH_IN,594209.730000,C1888557649,3151936.520000,3746146.250000,C1013511446,2540866.070000,1842864.220000,0,0,1.000000,0.188522,0.000000
83389,10,CASH_IN,103792.120000,C792601931,42696.840000,146488.970000,C1013511446,1946656.340000,1842864.220000,0,0,1.000000,2.430909,-0.010000
146724,12,CASH_OUT,398361.070000,C1402841545,8287.000000,0.000000,C1013511446,0.000000,2006826.390000,0,0,1.000000,48.070601,406648.070000
147414,12,TRANSFER,1608465.320000,C657294376,10678.000000,0.000000,C1013511446,398361.070000,2006826.390000,0,0,1.000000,150.633576,1619143.320000
201153,13,CASH_OUT,328650.740000,C47864924,151676.000000,0.000000,C1013511446,2006826.390000,2335477.130000,0,0,1.000000,2.166795,480326.740000
209103,13,TRANSFER,2657484.810000,C783323484,41409.000000,0.000000,C1013511446,2335477.130000,4992961.940000,0,0,1.000000,64.176503,2698893.810000
230719,14,CASH_IN,362466.680000,C1031467251,52474.000000,414940.680000,C1013511446,4992961.940000,4630495.260000,0,0,1.000000,6.907548,0.000000
299948,15,CASH_IN,236827.640000,C712201876,10927914.720000,11164742.360000,C1013511446,4630495.260000,4393667.620000,0,0,1.000000,0.021672,0.000000
342032,16,CASH_OUT,198969.470000,C1145539504,0.000000,0.000000,C1013511446,4393667.620000,4592637.080000,0,0,1.000000,nan,198969.470000


In [45]:
print("\n # Transactions which are Fraud")
print(len(df[df['isFraud']==1]))
print("\n % Transactions which are Fraud and New Balance Origination==0")
print(len(df[(df['isFraud']==1) & (df['newBalanceOrig']==0)])/len(df[df['isFraud']==1]))
print("\n # Transactions which are not Fraud")
print(len(df[df['isFraud']==0]))
print("\n % Transactions which are not Fraud and New Balance Origination==0")
print(len(df[(df['isFraud']==0) & (df['newBalanceOrig']==0)])/len(df[df['isFraud']==0]))
print("\n # Transactions which are not Fraud and Cash Out or Transfer")
print(len(df[(df['isFraud']==0) & (df['type'].isin(['CASH_OUT','TRANSFER']))]))
print("\n % Transactions which are not Fraud and Cash Out or Transfer and New Balance Origination==0")
print(len(df[(df['isFraud']==0) & (df['newBalanceOrig']==0) & (df['type'].isin(['CASH_OUT','TRANSFER']))])
      /len(df[(df['isFraud']==0) & (df['type'].isin(['CASH_OUT','TRANSFER']))]))


 # Transactions which are Fraud
8213

 % Transactions which are Fraud and New Balance Origination==0
0.9805186898818946

 # Transactions which are not Fraud
6354407

 % Transactions which are not Fraud and New Balance Origination==0
0.566774051457516

 # Transactions which are not Fraud and Cash Out or Transfer
2762196

 % Transactions which are not Fraud and Cash Out or Transfer and New Balance Origination==0
0.9009509100729999


#### Findings 18: New Balance Origination==0 is not a distinguisher between Fraud and Non Fraud transactions

In [46]:
df[df['amount']==0]

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud,week,amt_bal_ratio,diffBalanceOrig
2736447,212,CASH_OUT,0.000000,C1510987794,0.000000,0.000000,C1696624817,0.000000,0.000000,1,0,2.000000,nan,0.000000
3247298,250,CASH_OUT,0.000000,C521393327,0.000000,0.000000,C480398193,0.000000,0.000000,1,0,2.000000,nan,0.000000
3760289,279,CASH_OUT,0.000000,C539112012,0.000000,0.000000,C1106468520,538547.630000,538547.630000,1,0,2.000000,nan,0.000000
5563714,387,CASH_OUT,0.000000,C1294472700,0.000000,0.000000,C1325541393,7970766.570000,7970766.570000,1,0,3.000000,nan,0.000000
5996408,425,CASH_OUT,0.000000,C832555372,0.000000,0.000000,C1462759334,76759.900000,76759.900000,1,0,3.000000,nan,0.000000
5996410,425,CASH_OUT,0.000000,C69493310,0.000000,0.000000,C719711728,2921531.340000,2921531.340000,1,0,3.000000,nan,0.000000
6168500,554,CASH_OUT,0.000000,C10965156,0.000000,0.000000,C1493336195,230289.660000,230289.660000,1,0,3.000000,nan,0.000000
6205440,586,CASH_OUT,0.000000,C1303719003,0.000000,0.000000,C900608348,1328472.860000,1328472.860000,1,0,4.000000,nan,0.000000
6266414,617,CASH_OUT,0.000000,C1971175979,0.000000,0.000000,C1352345416,0.000000,0.000000,1,0,4.000000,nan,0.000000
6281483,646,CASH_OUT,0.000000,C2060908932,0.000000,0.000000,C1587892888,0.000000,0.000000,1,0,4.000000,nan,0.000000


#### Findings 19: All transactions with Amount==0 are Fraud

In [49]:
# Cases amount is positive but balances are zero
print("\n For only Transaction Types CASH_OUT & TRANSFER")
print("\n % Fraud Transactions where Amount>0 and Origin Balances are 0")
print(len(df[(df['isFraud']==1)&(df['amount']!=0)&(df['oldBalanceOrig']==0)&(df['newBalanceOrig']==0)])
      /len(df[(df['isFraud']==1)&(df['amount']!=0)]))
print("\n % Non Fraud Transactions where Amount>0 and Origin Balances are 0")
print(len(df[(df['isFraud']==0)&(df['amount']!=0)&(df['oldBalanceOrig']==0)
             &(df['newBalanceOrig']==0)&(df['type'].isin(['CASH_OUT','TRANSFER']))])
             /len(df[(df['isFraud']==0)&(df['amount']!=0)&(df['type'].isin(['CASH_OUT','TRANSFER']))]))
print("\n % Fraud Transactions where Amount>0 and Destination Balances are 0")
print(len(df[(df['isFraud']==1)&(df['amount']!=0)&(df['oldBalanceDest']==0)&(df['newBalanceDest']==0)])
     /len(df[(df['isFraud']==1)&(df['amount']!=0)]))
print("\n % Non Fraud Transactions where Amount>0 and Destination Balances are 0")
print(len(df[(df['isFraud']==0)&(df['amount']!=0)&(df['oldBalanceDest']==0)
             &(df['newBalanceDest']==0)&(df['type'].isin(['CASH_OUT','TRANSFER']))])
             /len(df[(df['isFraud']==0)&(df['amount']!=0)&(df['type'].isin(['CASH_OUT','TRANSFER']))]))


 For only Transaction Types CASH_OUT & TRANSFER

 % Fraud Transactions where Amount>0 and Origin Balances are 0
0.00304989630352568

 % Non Fraud Transactions where Amount>0 and Origin Balances are 0
0.4737321319703598

 % Fraud Transactions where Amount>0 and Destination Balances are 0
0.4965231182139807

 % Non Fraud Transactions where Amount>0 and Destination Balances are 0
0.0006176245277308345


#### Findings 20: Distinct difference in pattern of ZERO Origination and Destination Balance for Fraud and Non Fraud Transactions

In [58]:
print("\nOld Balance Orig Distribution for Fraud Transactions")
print(df[df['isFraud']==1]['oldBalanceOrig'].describe())
print("\nOld Balance Distribution Orig for Non Fraud Transactions")
print(df[(df['isFraud']==0)&(df['type'].isin(['CASH_OUT','TRANSFER']))]['oldBalanceOrig'].describe())


Old Balance Orig Distribution for Fraud Transactions
count       8213.000000
mean     1649667.605712
std      3547719.439109
min            0.000000
25%       125822.440000
50%       438983.450000
75%      1517771.480000
max     59585040.370000
Name: oldBalanceOrig, dtype: float64

Old Balance Distribution Orig for Non Fraud Transactions
count    2762196.000000
mean       42879.685563
std       135200.889180
min            0.000000
25%            0.000000
50%          283.000000
75%        30790.000000
max     43818855.300000
Name: oldBalanceOrig, dtype: float64


In [59]:
print("\nOld Balance Orig Distribution for Fraud Transactions for Old Balance Orig>0")
print(df[(df['isFraud']==1) & (df['oldBalanceOrig']>0)]['oldBalanceOrig'].describe())
print("\nOld Balance Orig Distribution for Non Fraud Transactions for Old Balance Orig>0")
print(df[(df['isFraud']==0)&(df['type'].isin(['CASH_OUT','TRANSFER'])) & (df['oldBalanceOrig']>0)]['oldBalanceOrig'].describe())


Old Balance Orig Distribution for Fraud Transactions for Old Balance Orig>0
count       8172.000000
mean     1657944.205300
std      3554679.214471
min           63.800000
25%       127564.330000
50%       443788.325000
75%      1532242.692500
max     59585040.370000
Name: oldBalanceOrig, dtype: float64

Old Balance Orig Distribution for Non Fraud Transactions for Old Balance Orig>0
count    1453655.000000
mean       81478.821277
std       177732.361829
min            0.050000
25%        10311.000000
50%        28690.000000
75%        85871.500000
max     43818855.300000
Name: oldBalanceOrig, dtype: float64


#### Findings 21: Old Balance Orig tend to be higher for Fraud than Non Fraud transactions. Customer with higher balances are usually targeted for Fraud

## Actions from Data Exploration Findings:

#### 1: Take first 3 weeks as Train and Week 4 as OOT Validation sample [Findings: 1]
#### 2: No missing value imputation required [Findings: 2]
#### 3: Rely of features created from single transactions and me vs others. Create a flag for nameOrig in consequetive 2 weeks[Findings: 3,4]
#### 4: Model data on only CASH_OUT and TRANSFER transactions/Create variable with extra weightage to CASH_OUT and TRANSFER transaction types [Findings: 5]
#### 5: isFlaggedFraud feature can be dropped [6 - 11]
#### 6: Create a flag for Payments to Merchants if modelling data on all transaction types[12 - 14]
#### 7: Assign a standard value for Amount/oldBalanceOrig for CASH IN type transactions [15-17]
#### 8: Assign Flag for Amount==0 [19]
#### 9: Assign Flag for CASH_OUT & TRANSFER type transactions when Amount>0 and newBalanceOrig==oldBalanceOrig==0 
#### 10: Assign Flag for CASH_OUT & TRANSFER type transactions when Amount>0 and newBalanceDest==oldBalanceDest==0 
#### 11: Make buckets for Old Balance Orig for CASH_OUT & TRANSFER type transactions [21]